## 0.0 Impots

In [1]:
import json
import os
import random

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import (
    ElasticNet,
    Lasso,
    LinearRegression,
    RANSACRegressor,
    Ridge,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor

## 0.1 Loading Dataset

In [2]:
path_home = os.path.dirname(os.getcwd())
# Dados de Treino
path_X_traning = os.path.join(path_home, 'data', 'X_training.csv')
path_y_traning = os.path.join(path_home, 'data', 'y_training.csv')

x_training  = pd.read_csv(path_X_traning)
y_training = pd.read_csv(path_y_traning)

# Dados de Validação
path_X_validation = os.path.join(path_home, 'data', 'X_validation.csv')
path_y_validation = os.path.join(path_home, 'data', 'y_validation.csv')

x_validation  = pd.read_csv(path_X_validation)
y_validation = pd.read_csv(path_y_validation)

# Dados de Teste
path_X_test = os.path.join(path_home, 'data', 'X_test.csv')
path_y_test = os.path.join(path_home, 'data', 'y_test.csv')

x_test  = pd.read_csv(path_X_test)
y_test = pd.read_csv(path_y_test)

## 0.2 Helpe Function

In [3]:
def get_metrics(y_data, yhat_model):
    metrics = {
        'R2': r2_score(y_data, yhat_model),
        'MSE': mean_squared_error(y_data, yhat_model),
        'RMSE': root_mean_squared_error(y_data, yhat_model),
        'MAE': mean_absolute_error(y_data, yhat_model),
        'MAPE': mean_absolute_percentage_error(y_data, yhat_model),

    }
    return metrics
    
def classifier_evaluation(model_classifier, param, data):
    metric = pd.DataFrame()
    
    data_mapping = {
        'training': (x_training, y_training),
        'validation': (x_validation, y_validation),
        'test': (np.concatenate((x_training, x_validation)), np.concatenate((y_training, y_validation)))
    }

    x_data, y_data = data_mapping.get(data, (None, None))

    if x_data is None or y_data is None:
        raise ValueError("Invalid data type provided. Choose from 'training', 'validation', or 'test'.")

    for i in range(1, len(param) + 1):
        model = model_classifier(**param)
        model.fit(x_data, y_data.values.ravel())

        yhat_model = model.predict(x_data)

        metrics_mapping = {
            'training': y_training,
            'validation': y_validation,
            'test': y_test
        }

        metrics = get_metrics(metrics_mapping[data], yhat_model)

        metric = pd.DataFrame({
            'name': model.__class__.__name__,
            **metrics,
            'param': json.dumps(param)
        }, index=[0])

    return metric

def classifier_evaluation_poli(model_classifier, param, dataset, data):
    x_data, y_data = dataset['x'], dataset['y']
    metric = pd.DataFrame()

    for i in range(1, len(param)+1):
        model = model_classifier(**param)
        model.fit(x_data, y_data.values.ravel()) 

        yhat_model = model.predict(x_data)

        metrics_mapping = {
            'training': y_training,
            'validation': y_validation,
            'test': y_test
        }
        
        metrics = get_metrics(metrics_mapping[data], yhat_model)
        metric = pd.DataFrame({
            'name': model.__class__.__name__ ,
            **metrics,
            'param': json.dumps(param)
        }, index=[0])
    
    return metric

# 1.0 Training

In [4]:
data = 'training'

## 1.1 Linear Regresson

In [5]:
resul_lr = pd.DataFrame()
model = LinearRegression
param = {
        'fit_intercept': True
    }

result = classifier_evaluation(model, param, data)

resul_lr = pd.concat([resul_lr,result]).reset_index(drop=True)

In [6]:
resul_lr

,name,R2,MSE,RMSE,MAE,MAPE,param
0,LinearRegression,0.046058,455.996112,21.354065,16.998249,8.653186,"{""fit_intercept"": true}"


## 1.2 Linear Regresson Lasso

In [7]:
resul_ls = pd.DataFrame()
model = Lasso
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_ls = pd.concat([resul_ls,result]).reset_index(drop=True)

In [8]:
resul_ls

,name,R2,MSE,RMSE,MAE,MAPE,param
0,Lasso,0.000000,478.012560,21.863498,17.365090,8.742213,"{""alpha"": 3, ""max_iter"": 1500}"
1,Lasso,0.007401,474.474834,21.782443,17.305484,8.736697,"{""alpha"": 1, ""max_iter"": 1500}"
2,Lasso,0.001125,477.474834,21.851198,17.355395,8.741522,"{""alpha"": 2, ""max_iter"": 1000}"
3,Lasso,0.007401,474.474834,21.782443,17.305484,8.736697,"{""alpha"": 1, ""max_iter"": 2000}"
4,Lasso,0.000000,478.012560,21.863498,17.365090,8.742213,"{""alpha"": 3, ""max_iter"": 1500}"
5,Lasso,0.007401,474.474834,21.782443,17.305484,8.736697,"{""alpha"": 1, ""max_iter"": 1500}"
6,Lasso,0.000000,478.012560,21.863498,17.365090,8.742213,"{""alpha"": 3, ""max_iter"": 2000}"
7,Lasso,0.007401,474.474834,21.782443,17.305484,8.736697,"{""alpha"": 1, ""max_iter"": 2000}"
8,Lasso,0.007401,474.474834,21.782443,17.305484,8.736697,"{""alpha"": 1, ""max_iter"": 2000}"
9,Lasso,0.000000,478.012560,21.863498,17.365090,8.742213,"{""alpha"": 3, ""max_iter"": 2000}"


## 1.3 Linear Regresson Ridge

In [9]:
resul_lrr = pd.DataFrame()
model = Ridge
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lrr = pd.concat([resul_lrr,result]).reset_index(drop=True)

In [10]:
resul_lrr

,name,R2,MSE,RMSE,MAE,MAPE,param
0,Ridge,0.046056,455.997238,21.354092,16.998366,8.653638,"{""alpha"": 2, ""max_iter"": 2000}"
1,Ridge,0.046053,455.998584,21.354123,16.998441,8.653856,"{""alpha"": 3, ""max_iter"": 1000}"
2,Ridge,0.046056,455.997238,21.354092,16.998366,8.653638,"{""alpha"": 2, ""max_iter"": 1500}"
3,Ridge,0.046058,455.996401,21.354072,16.998308,8.653415,"{""alpha"": 1, ""max_iter"": 1000}"
4,Ridge,0.046053,455.998584,21.354123,16.998441,8.653856,"{""alpha"": 3, ""max_iter"": 1500}"
5,Ridge,0.046056,455.997238,21.354092,16.998366,8.653638,"{""alpha"": 2, ""max_iter"": 1500}"
6,Ridge,0.046058,455.996401,21.354072,16.998308,8.653415,"{""alpha"": 1, ""max_iter"": 2000}"
7,Ridge,0.046056,455.997238,21.354092,16.998366,8.653638,"{""alpha"": 2, ""max_iter"": 2000}"
8,Ridge,0.046058,455.996401,21.354072,16.998308,8.653415,"{""alpha"": 1, ""max_iter"": 1500}"
9,Ridge,0.046058,455.996401,21.354072,16.998308,8.653415,"{""alpha"": 1, ""max_iter"": 1000}"


## 1.4 Linear Regresson ElasticNet

In [11]:
resul_lre = pd.DataFrame()
model = ElasticNet
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000],
        'l1_ratio': [0.3, 0.5, 0.7]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter']),
        'l1_ratio': random.choice(para['l1_ratio'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lre = pd.concat([resul_lre,result]).reset_index(drop=True)

In [12]:
resul_lre

,name,R2,MSE,RMSE,MAE,MAPE,param
0,ElasticNet,0.000141,477.945109,21.861956,17.363748,8.742126,"{""alpha"": 3, ""max_iter"": 1000, ""l1_ratio"": 0.7}"
1,ElasticNet,0.008744,473.833027,21.767706,17.290950,8.727685,"{""alpha"": 1, ""max_iter"": 1500, ""l1_ratio"": 0.3}"
2,ElasticNet,0.008744,473.833027,21.767706,17.290950,8.727685,"{""alpha"": 1, ""max_iter"": 1000, ""l1_ratio"": 0.3}"
3,ElasticNet,0.004940,475.651113,21.809427,17.321482,8.738012,"{""alpha"": 2, ""max_iter"": 1500, ""l1_ratio"": 0.3}"
4,ElasticNet,0.007832,474.268889,21.777715,17.299507,8.732300,"{""alpha"": 1, ""max_iter"": 1500, ""l1_ratio"": 0.5}"
5,ElasticNet,0.007832,474.268889,21.777715,17.299507,8.732300,"{""alpha"": 1, ""max_iter"": 1000, ""l1_ratio"": 0.5}"
6,ElasticNet,0.008744,473.833027,21.767706,17.290950,8.727685,"{""alpha"": 1, ""max_iter"": 2000, ""l1_ratio"": 0.3}"
7,ElasticNet,0.007832,474.268889,21.777715,17.299507,8.732300,"{""alpha"": 1, ""max_iter"": 1000, ""l1_ratio"": 0.5}"
8,ElasticNet,0.004368,475.924363,21.815691,17.326820,8.739289,"{""alpha"": 2, ""max_iter"": 2000, ""l1_ratio"": 0.5}"
9,ElasticNet,0.003207,476.479361,21.828407,17.337170,8.740140,"{""alpha"": 3, ""max_iter"": 2000, ""l1_ratio"": 0.3}"


## 1.5 Decision Tree

In [13]:
resul_dt = pd.DataFrame()
model = DecisionTreeRegressor
for i in range(2, 20, 1):
    param = {
            'max_depth': i
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [14]:
resul_dt

,name,R2,MSE,RMSE,MAE,MAPE,param
0,DecisionTreeRegressor,0.043334,457.298367,21.384536,16.986509,8.500932,"{""max_depth"": 2}"
1,DecisionTreeRegressor,0.063005,447.895381,21.163539,16.833243,8.415119,"{""max_depth"": 3}"
2,DecisionTreeRegressor,0.086649,436.593329,20.894816,16.618123,8.268743,"{""max_depth"": 4}"
3,DecisionTreeRegressor,0.113523,423.747268,20.585122,16.368766,7.869536,"{""max_depth"": 5}"
4,DecisionTreeRegressor,0.144608,408.887912,20.220977,16.026872,7.307613,"{""max_depth"": 6}"
5,DecisionTreeRegressor,0.191565,386.442081,19.658130,15.505405,6.535123,"{""max_depth"": 7}"
6,DecisionTreeRegressor,0.245649,360.589105,18.989184,14.842361,6.150264,"{""max_depth"": 8}"
7,DecisionTreeRegressor,0.310891,329.402526,18.149450,13.964753,5.608744,"{""max_depth"": 9}"
8,DecisionTreeRegressor,0.384624,294.157341,17.151016,12.925051,4.871411,"{""max_depth"": 10}"
9,DecisionTreeRegressor,0.459861,258.193172,16.068390,11.749243,4.169772,"{""max_depth"": 11}"


## 1.6 Random Forest

In [15]:
resul_dt = pd.DataFrame()
model = RandomForestRegressor
para = {
        'n_estimators': [100, 200, 300],
    }
for i in range(2, 20, 1):
    param = {
            'max_depth': i,
            'n_estimators': random.choice(para['n_estimators'])
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

KeyboardInterrupt: 

In [16]:
resul_dt

,name,R2,MSE,RMSE,MAE,MAPE,param
0,RandomForestRegressor,0.049885,454.166691,21.311187,16.930611,8.524453,"{""max_depth"": 2, ""n_estimators"": 200}"
1,RandomForestRegressor,0.077580,440.928554,20.998299,16.687064,8.436117,"{""max_depth"": 3, ""n_estimators"": 100}"
2,RandomForestRegressor,0.106761,426.979694,20.663487,16.421102,8.294292,"{""max_depth"": 4, ""n_estimators"": 200}"
3,RandomForestRegressor,0.141486,410.380312,20.257846,16.093608,8.043305,"{""max_depth"": 5, ""n_estimators"": 200}"
4,RandomForestRegressor,0.185376,389.400532,19.733234,15.686169,7.713385,"{""max_depth"": 6, ""n_estimators"": 100}"
5,RandomForestRegressor,0.243695,361.523476,19.013771,15.127177,7.285362,"{""max_depth"": 7, ""n_estimators"": 300}"
6,RandomForestRegressor,0.311443,329.139090,18.142191,14.426653,6.795823,"{""max_depth"": 8, ""n_estimators"": 300}"
7,RandomForestRegressor,0.383591,294.651211,17.165407,13.660254,6.300257,"{""max_depth"": 9, ""n_estimators"": 200}"
8,RandomForestRegressor,0.466579,254.981722,15.968147,12.696189,5.790455,"{""max_depth"": 10, ""n_estimators"": 200}"
9,RandomForestRegressor,0.547126,216.479458,14.713241,11.689069,5.193006,"{""max_depth"": 11, ""n_estimators"": 300}"


## 1.7 Polynomial Features

In [ ]:
resul_pf = pd.DataFrame()
model = LinearRegression

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_training)
    data_poly = {'x': X_poly, 'y': y_training}

    param = {
        'fit_intercept': True,
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pf = pd.concat([resul_pf,result]).reset_index(drop=True)

In [19]:
resul_pf

,name,R2,MSE,RMSE,MAE,MAPE,param
0,LinearRegression,0.7253,131.310015,11.459058,7.266166,2.215335,"{""fit_intercept"": true}"


## 1.8 Polynomial Features Lasso

In [17]:
resul_pfl = pd.DataFrame()
model = Lasso
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_training)
    data_poly = {'x': X_poly, 'y': y_training}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pfl = pd.concat([resul_pfl, result]).reset_index(drop=True)

/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.303e+06, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.303e+06, tolerance: 5.042e+02
  model = cd_fast.enet_coordinate_descent(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to 

In [ ]:
resul_pfl

## 1.9 Polynomial Features Ridge

In [ ]:
resul_pfr = pd.DataFrame()
model = Ridge
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_training)
    data_poly = {'x': X_poly, 'y': y_training}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pfr = pd.concat([resul_pfr, result]).reset_index(drop=True)

In [ ]:
resul_pfr

## 1.10 Polynomial Features Elastic Net

In [ ]:
resul_pfe = pd.DataFrame()
model = ElasticNet
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_training)
    data_poly = {'x': X_poly, 'y': y_training}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pfe = pd.concat([resul_pfe, result]).reset_index(drop=True)

In [ ]:
resul_pfe

# 2.0 Validation

In [ ]:
data = 'validation'

## 2.1 Linear Regresson

In [ ]:
resul_lr = pd.DataFrame()
model = LinearRegression
param = {
        'fit_intercept': True
    }

result = classifier_evaluation(model, param, data)

resul_lr = pd.concat([resul_lr,result]).reset_index(drop=True)

In [ ]:
resul_lr

## 2.2 Linear Regresson Lasso

In [ ]:
resul_ls = pd.DataFrame()
model = Lasso
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_ls = pd.concat([resul_ls,result]).reset_index(drop=True)

In [ ]:
resul_ls

## 2.3 Linear Regresson Ridge

In [ ]:
resul_lrr = pd.DataFrame()
model = Ridge
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lrr = pd.concat([resul_lrr,result]).reset_index(drop=True)

In [ ]:
resul_lrr

## 2.4 Linear Regresson ElasticNet

In [ ]:
resul_lre = pd.DataFrame()
model = ElasticNet
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000],
        'l1_ratio': [0.3, 0.5, 0.7]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter']),
        'l1_ratio': random.choice(para['l1_ratio'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lre = pd.concat([resul_lre,result]).reset_index(drop=True)

In [ ]:
resul_lre

## 2.5 Decision Tree

In [ ]:
resul_dt = pd.DataFrame()
model = DecisionTreeRegressor
for i in range(2, 20, 1):
    param = {
            'max_depth': i
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [ ]:
resul_dt

## 2.6 Random Forest

In [ ]:
resul_dt = pd.DataFrame()
model = RandomForestRegressor
para = {
        'n_estimators': [100, 200, 300],
    }
for i in range(2, 20, 1):
    param = {
            'max_depth': i,
            'n_estimators': random.choice(para['n_estimators'])
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [ ]:
resul_dt

## 2.7 Polynomial Features

In [ ]:
resul_pf = pd.DataFrame()
model = LinearRegression

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'fit_intercept': True,
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pf = pd.concat([resul_pf,result]).reset_index(drop=True)

In [ ]:
resul_pf

## 2.8 Polynomial Features Lasso

In [ ]:
resul_pfl = pd.DataFrame()
model = Lasso
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfl = pd.concat([resul_pfl, result]).reset_index(drop=True)

In [ ]:
resul_pfl

## 2.9 Polynomial Features Ridge

In [ ]:
resul_pfr = pd.DataFrame()
model = Ridge
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfr = pd.concat([resul_pfr, result]).reset_index(drop=True)

In [ ]:
resul_pfr

## 2.10 Polynomial Features Elastic Net

In [ ]:
resul_pfe = pd.DataFrame()
model = ElasticNet
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfe = pd.concat([resul_pfe, result]).reset_index(drop=True)

In [ ]:
resul_pfe

# 3.0 Test

In [ ]:
data = 'Test'

## 3.1 Linear Regresson

In [ ]:
resul_lr = pd.DataFrame()
model = LinearRegression
param = {
        'fit_intercept': True
    }

result = classifier_evaluation(model, param, data)

resul_lr = pd.concat([resul_lr,result]).reset_index(drop=True)

In [ ]:
resul_lr

## 3.2 Linear Regresson Lasso

In [ ]:
resul_ls = pd.DataFrame()
model = Lasso
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_ls = pd.concat([resul_ls,result]).reset_index(drop=True)

In [ ]:
resul_ls

## 3.3 Linear Regresson Ridge

In [ ]:
resul_lrr = pd.DataFrame()
model = Ridge
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lrr = pd.concat([resul_lrr,result]).reset_index(drop=True)

In [ ]:
resul_lrr

## 3.4 Linear Regresson ElasticNet

In [ ]:
resul_lre = pd.DataFrame()
model = ElasticNet
para = {
        'alpha': [1, 2, 3],
        'max_iter': [1000, 1500, 2000],
        'l1_ratio': [0.3, 0.5, 0.7]
    }

for i in range(2, 20, 1):
    param = {
        'alpha': random.choice(para['alpha']),
        'max_iter': random.choice(para['max_iter']),
        'l1_ratio': random.choice(para['l1_ratio'])
    }

    result = classifier_evaluation(model, param, data)

    resul_lre = pd.concat([resul_lre,result]).reset_index(drop=True)

In [ ]:
resul_lre

## 3.5 Decision Tree

In [ ]:
resul_dt = pd.DataFrame()
model = DecisionTreeRegressor
for i in range(2, 20, 1):
    param = {
            'max_depth': i
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [ ]:
resul_dt

## 3.6 Random Forest

In [ ]:
resul_dt = pd.DataFrame()
model = RandomForestRegressor
para = {
        'n_estimators': [100, 200, 300],
    }
for i in range(2, 20, 1):
    param = {
            'max_depth': i,
            'n_estimators': random.choice(para['n_estimators'])
        }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [ ]:
resul_dt

## 3.7 Polynomial Features

In [ ]:
resul_pf = pd.DataFrame()
model = LinearRegression

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'fit_intercept': True,
    }

    result = classifier_evaluation_poli(model, param, data_poly, data)

    resul_pf = pd.concat([resul_pf,result]).reset_index(drop=True)

In [ ]:
resul_pf

## 3.8 Polynomial Features Lasso

In [ ]:
resul_pfl = pd.DataFrame()
model = Lasso
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfl = pd.concat([resul_pfl, result]).reset_index(drop=True)

In [ ]:
resul_pfl

## 3.9 Polynomial Features Ridge

In [ ]:
resul_pfr = pd.DataFrame()
model = Ridge
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfr = pd.concat([resul_pfr, result]).reset_index(drop=True)

In [ ]:
resul_pfr

## 3.10 Polynomial Features Elastic Net

In [ ]:
resul_pfe = pd.DataFrame()
model = ElasticNet
para = {
        'max_iter': [1000, 1500, 2000],
        'alpha': [1, 2, 3]
    }

for i in range(2, 5, 1):
    poly = PolynomialFeatures(degree=i)
    X_poly = poly.fit_transform(x_validation)
    data_poly = {'x': X_poly, 'y': y_validation}

    param = {
        'alpha': i,
        'max_iter': random.choice(para['max_iter'])
    }

    result = classifier_evaluation(model, param, data_poly)

    resul_pfe = pd.concat([resul_pfe, result]).reset_index(drop=True)

In [ ]:
resul_pfe